# Getting Started with the Agent Development Kit (ADK) with Neo4j

This notebook shows how to use the Vertex AI Agent Framework to setup a multi-agent system integrating Neo4j.

The Vertex AI ADK is a python-based SDK that empowers developers to build multi-agent applications with custom logic, tools, and integrations.

In this notebook you will learn how to do the following:

*  Install Agent Development Kit
*  Define functions as tools
*  Define a database agent and an investor relationship agent
*  Create a "Investment Research" Multi-Agent


## Learn more about Neo4j

* https://neo4j.com/genai
* https://neo4j.com/developer
* https://graphrag.com
* https://neo4j.com/labs/genai-ecosystem
* [Neo4j and MCP Toolbox](https://neo4j.com/blog/developer/ai-agents-gen-ai-toolbox/)

## Learn more about the ADK

* [Launch Blog](https://cloud.google.com/blog/products/ai-machine-learning/build-and-manage-multi-system-agents-with-vertex-ai)
* [Developer Blog](https://developers.googleblog.com/en/agent-development-kit-easy-to-build-multi-agent-applications/)
* https://pypi.org/project/google-adk/
* https://google.github.io/adk-docs/
* https://github.com/google/adk-samples/
* [LLM Agents](https://google.github.io/adk-docs/agents/llm-agents/#putting-it-together-example)
* [Deploy to Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/)

## Company News Knowledge Graph

Our example dataset will be a small subset (250k entities) of [diffbot's](https://diffbot.com/) global Knowledge Graph (50bn entities).

It contains organizations, people in their leadership, locations and industries.

Additionally articles mentioning these companies, which are chunked and indexed as vector embeddings.

![](https://camo.githubusercontent.com/2ebaff2ceb74cd8af8f7ef8a5a4791ef92ea8f631c0fc41d828eb8ae9c7bf553/68747470733a2f2f692e696d6775722e636f6d2f6c574a5a5345652e706e67)

You can access and query a read only version of that graph with the following credentials:

* URL: https://demo.neo4jlabs.com:7473
* Username: companies
* Password: companies
* Database: companies

# Authenticate
You need this to download the SDK from GCS

In [1]:
from google.colab import auth
auth.authenticate_user()

# Install the Agent Development Kit

In [2]:
!pip3 install --upgrade --quiet google-adk neo4j-rust-ext

In [3]:
import os
import random
import sys

# Developer API or Vertex AI

The ADK supports two APIs:

* Google Gemini API (AI Studio)
* Vertex AI Gemini API.

In [4]:
# Only run this block for ML Developer API. Use your own API key.
import os

GOOGLE_API_KEY = "FILL YOUR API KEY" #@param {type:"string"}

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "0"
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [5]:
# Only run this block for Vertex AI API Use your own project / location.
import os

GOOGLE_CLOUD_PROJECT = "vertex-ai-neo4j-extension" #@param {type:"string"}

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

# Neo4j Database Connectivity

We define a small class `neo4jDatabase` that manages connecting to Neo4j and running read queries.

In [6]:
import logging

logger = logging.getLogger('agent_neo4j_cypher')
logger.info("Initializing Database for tools")

import logging

logging.getLogger("neo4j").setLevel(logging.ERROR)
logging.getLogger("google_genai").setLevel(logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="google_genai.types")
warnings.filterwarnings("ignore", category=UserWarning, module="neo4j.notifications")


In [7]:
from neo4j import GraphDatabase
from typing import Any
import re

class neo4jDatabase:
    def __init__(self,  neo4j_uri: str, neo4j_username: str, neo4j_password: str):
        """Initialize connection to the Neo4j database"""
        logger.debug(f"Initializing database connection to {neo4j_uri}")
        d = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))
        d.verify_connectivity()
        self.driver = d

    def is_write_query(self, query: str) -> bool:
      return re.search(r"\b(MERGE|CREATE|SET|DELETE|REMOVE|ADD)\b", query, re.IGNORECASE) is not None

    def _execute_query(self, query: str, params: dict[str, Any] | None = None) -> list[dict[str, Any]]:
        """Execute a Cypher query and return results as a list of dictionaries"""
        logger.debug(f"Executing query: {query}")
        try:
            if self.is_write_query(query):
                logger.error(f"Write query not supported {query}")
                raise "Write Queries are not supported in this agent"
                # logger.debug(f"Write query affected {counters}")
                # result = self.driver.execute_query(query, params)
                # counters = vars(result.summary.counters)
                # return [counters]
            else:
                result = self.driver.execute_query(query, params)
                results = [dict(r) for r in result.records]
                logger.debug(f"Read query returned {len(results)} rows")
                return results
        except Exception as e:
            logger.error(f"Database error executing query: {e}\n{query}")
            raise

In [8]:
db = neo4jDatabase("neo4j+s://demo.neo4jlabs.com","companies","companies")

In [9]:
# Testing database connection
db._execute_query("RETURN 1")

[{'1': 1}]

## Defining Functions for our Database Agent

* get_schema - Retrieve Database Schema for the LLM
* execute_read_query - Execute Cypher Read Query

In [10]:
def get_schema() -> list[dict[str,Any]]:
  """Get the schema of the database, returns node-types(labels) with their types and attributes and relationships between node-labels
  Args: None
  Returns:
    list[dict[str,Any]]: A list of dictionaries representing the schema of the database
    For example
    ```
    [{'label': 'Person','attributes': {'summary': 'STRING','id': 'STRING unique indexed', 'name': 'STRING indexed'},
      'relationships': {'HAS_PARENT': 'Person', 'HAS_CHILD': 'Person'}}]
    ```
  """
  try:
      results = db._execute_query(
              """
call apoc.meta.data() yield label, property, type, other, unique, index, elementType
where elementType = 'node' and not label starts with '_'
with label,
collect(case when type <> 'RELATIONSHIP' then [property, type + case when unique then " unique" else "" end + case when index then " indexed" else "" end] end) as attributes,
collect(case when type = 'RELATIONSHIP' then [property, head(other)] end) as relationships
RETURN label, apoc.map.fromPairs(attributes) as attributes, apoc.map.fromPairs(relationships) as relationships
              """
          )
      return results
  except Exception as e:
      return [{"error":str(e)}]

In [11]:
# get_schema()

In [12]:
def execute_read_query(query: str, params: dict[str, Any]) -> list[dict[str, Any]]:
    """
    Execute a Neo4j Cypher query and return results as a list of dictionaries
    Args:
        query (str): The Cypher query to execute
        params (dict[str, Any], optional): The parameters to pass to the query or None.
    Raises:
        Exception: If there is an error executing the query
    Returns:
        list[dict[str, Any]]: A list of dictionaries representing the query results
    """
    try:
        if params is None:
            params = {}
        results = db._execute_query(query, params)
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [13]:
execute_read_query("RETURN 1", None)

[{'1': 1}]

## Defining Functions for our Investment Research Agent

* get_schema - Retrieve Database Schema for the LLM
* get_investors - Returns the investor in the company with this name or id.

In [14]:
def get_investors(company: str) -> list[dict[str, Any]]:
    """
    Returns the investor in the company with this name or id.
    Args:
        company (str): name of the company to find investors in
    Returns:
        list[dict[str, Any]]: A list of investor ids, names (and their types Organization or Person)
    """
    try:
        results = db._execute_query("""
        MATCH p=(o:Organization)<-[r:HAS_INVESTOR]-(i)
        WHERE o.name=$company OR o.id=$company
        RETURN i.id as id, i.name as name, head(labels(i)) as type
        """, {"company":company})
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [15]:
get_investors("Cloudflare")

[{'id': 'EIBehdsQ4ME-yr8WK8tiC1w',
  'name': 'Atlas Financial Group',
  'type': 'Organization'},
 {'id': 'E-qHiBRsMNb2wOsqdSx2CfQ',
  'name': 'Pelion Venture Partners',
  'type': 'Organization'},
 {'id': 'E7obn7yRnMP6HjTQ4xOL6Dg', 'name': 'Venrock', 'type': 'Organization'},
 {'id': 'EfCgjTBhgNdyHkynILGypYg',
  'name': 'New Enterprise Associates (NEA)',
  'type': 'Organization'},
 {'id': 'EptIVnF6gNuOmOmAdBku7Uw',
  'name': 'National Science Foundation (NSF)',
  'type': 'Organization'},
 {'id': 'EdE6JQUuvMf6rTFv2qMJZqQ',
  'name': 'Union Square Ventures',
  'type': 'Organization'},
 {'id': 'E0aVDBaUAMSWgjywIrE4IwA',
  'name': 'Greenspring Associates',
  'type': 'Organization'},
 {'id': 'EThF8yM26NOmTZYsND2DqRQ',
  'name': 'Microsoft Accelerator',
  'type': 'Organization'},
 {'id': 'EUFq-3WlpNsq0pvfUYWXOEA', 'name': 'Google', 'type': 'Organization'},
 {'id': 'EmuHMD_KVM1OUKHkQE21s5A',
  'name': 'Qualcomm Ventures',
  'type': 'Organization'},
 {'id': 'E1XokcZs2NqGwLR47G92lFQ', 'name': 'Ba

# Define our Agents

An AI agent reasons, plans, and takes actions.

The agent takes actions via access to **tools**, deciding how and when to invoke a tool. The agent also manages orchestration, creating a plan for answering a user query and adapting to responses that aren't quite correct.

Agent **tools** can be Python functions, or Vertex AI Extensions, or MCP Servers.

## Creating the Database Agent.

* display name
* instructions - detailed, making it clear exactly how the agent should behave and which tools to use when
* tools



In [16]:
from google.adk.agents import Agent

In [17]:
MODEL="gemini-2.5-pro-exp-03-25"

In [40]:
database_agent = Agent(
    model=MODEL,
    name='graph_database_agent',
    instruction="""
      You are an Neo4j graph database and Cypher query expert, that must use the database schema with a user question and repeatedly generate valid cypher statements
      to execute on the database and answer the user's questions in a friendly manner in natural language.
      If in doubt the database schema is always prioritized when it comes to nodes-types (labels) or relationship-types or property names, never take the user's input at face value.
      If the user requests also render tables, charts or other artifacts with the query results.
      Always validate the correct node-labels at the end of a relationship based on the schema.

      If a query fails or doesn't return data, use the error response 3 times to try to fix the generated query and re-run it, don't return the error to the user.
      If you cannot fix the query, explain the issue to the user and apologize.
      *You are prohibited* from using directional arrows (like -> or <-) in the graph patterns, always use undirected patterns like `(:Label)-[:TYPE]-(:Label)`.
      You get negative points for using directional arrays in patterns.

      Fetch the graph database schema first and keep it in session memory to access later for query generation.
      Keep results of previous executions in session memory and access if needed, for instance ids or other attributes of nodes to find them again
      removing the need to ask the user. This also allows for generating shorter, more focused and less error-prone queries
      to for drill downs, sequences and loops.
      If possible resolve names to primary keys or ids and use those for looking up entities.
      The schema always indicates *outgoing* relationship-types from an entity to another entity, the graph patterns read like english language.
      `company has supplier` would be the pattern `(o:Organization)-[:HAS_SUPPLIER]-(s:Organization)`

      To get the schema of a database use the `get_schema` tool without parameters. Store the response of the schema tool in session context
      to access later for query generation.

      To answer a user question generate one or more Cypher statements based on the database schema and the parts of the user question.
      If necessary resolve categorical attributes (like names, countries, industries, publications) first by retrieving them for a set of entities to translate from the user's request.
      Use the `execute_query` tool repeatedly with the Cypher statements, you MUST generate statements that use named query parameters with `$parameter` style names
      and MUST pass them as a second dictionary parameter to the tool, even if empty.
      Parameter data can come from the users requests, prior query results or additional lookup queries.
      After the data for the question has been sufficiently retrieved, pass the data and control back to the parent agent.
    """,
    tools=[
        get_schema, execute_read_query
    ]
)

In [41]:
investor_research_agent = Agent(
    model=MODEL,
    name='investment_research_agent',
    instruction="""
    You are an agent that has access to a database of investment relationships between companies and indivduals.
    Use the get_investors tool when asked to find the investors of a company by id and name.
    If you do so, try to always return not just the factual attribute data but also
    investor ids to allow the other agents to investigate them more.
    """,
    tools=[
        get_schema, get_investors
    ]
)

## Model Context Protocol (MCP) with MCP Toolbox

With the [Neo4j Integration in Google's MCP Toolbox](https://googleapis.github.io/genai-toolbox/resources/tools/neo4j-cypher/) we can quickly declaratively define sources and tools backed by database queries and deploy that toolset to Cloud Run.

Our [tool definitions are here](https://github.com/neo4j-examples/neo4j-gcp-vertex-ai-langchain/blob/main/toolbox-companies/tools.yaml) for these tools:

* `industries` - List of Industry names
* `companies` - List of Companies (id, name, summary) by fulltext search
* `companies_in_industry` - Companies (id, name, summary) in a given industry by industry
* `articles_in_month` - List of Articles (id, author, title, date, sentiment) in a month timeframe from the given date * (yyyy-mm-dd)
* `article` - Single Article details (id, author, title, date, sentiment, site, summary, content) by article id
* `companies_in_articles` - Companies (id, name, summary) mentioned in articles by list of article ids
* `people_at_company` - People (name, role) associated with a company by company id


**Now** we can use the ADK's new MCP wrappers which exist for STDIO and HTTP/SSE MCP servers to make these MCP Server tools available to one of our agents.

In [39]:
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, SseServerParams

In [42]:
async def load_tools():
    async with MCPToolset( connection_params=SseServerParams(url="https://toolbox-990868019953.us-central1.run.app/mcp/sse")) as toolset:
      tools = await toolset.load_tools()
    return tools

In [43]:
tools = await load_tools()
tools.extend([get_schema])

In [44]:
investment_research_agent = Agent(
    model=MODEL,
    name='investment_research_agent',
    instruction="""
    You are an agent that has access to a knowledge graph of companies (organizations), people involved with them, articles about companies,
    and industry categories and technologies.
    You have a set of tools to access different aspects of the investment database.
    You will be tasked by other agents to fetch certain information from that knowledge graph.
    If you do so, try to always return not just the factual attribute data but also
    ids of companies, articles, people to allow the other tools to investigate them more.
    """,
    tools=tools
)

In [45]:
root_agent = Agent(
    model=MODEL,
    name='investment_agent',
    global_instruction = "",
    instruction="""
    You are an agent that has access to a knowledge graph of companies (organizations), people involved with them, articles about companies,
    and industry categories and technologies.
    You have a set of agents to retrieve information from that knowledge graph, if possible prefer the research agents over the database agent.
    If the user requests it, do render tables, charts or other artifacts with the research results.
    """,

    sub_agents=[investor_research_agent, investment_research_agent, database_agent]
)

# Let's try it

You can [run the ADK locally](https://google.github.io/adk-docs/get-started/local-testing/#expected-output) a web application with `adk web` or the FastAPI server with `adk api_server`.

You can also deploy the agent to [Cloud Run](https://google.github.io/adk-docs/deploy/cloud-run/) or to [Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/).

In [46]:
APP_NAME = 'Neo4j Investment Researcher'
USER_ID = 'Michael Hunger'

from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent


runner = InMemoryRunner(app_name=APP_NAME, agent=root_agent)

session = runner.session_service.create_session( app_name=runner.app_name, user_id=USER_ID)

async def run_prompt(new_message: str):
  content = UserContent(parts=[Part(text=new_message)])
# print (content)
  result = None
  async for event in runner.run_async(user_id=session.user_id, session_id=session.id, new_message=content):
#    print(event.content.model_dump(exclude_none=True))
#    print(event.content.parts)
    for part in event.content.parts:
      print(part.text, part.function_call, part.function_response)
      if part.text:
#        print(part.text)
        result = part.text
  return result

In [47]:
await run_prompt('How many people are in the database?')


None id='adk-af40c5f1-b747-4b01-a3ea-c1c7521c2ace' args={'agent_name': 'graph_database_agent'} name='transfer_to_agent' None
None None id='adk-af40c5f1-b747-4b01-a3ea-c1c7521c2ace' name='transfer_to_agent' response={}
Okay, I can help with that. First, I need to check the database schema to see how people are represented.

 None None
None id='adk-03f904c5-cb65-47f5-9cda-a132eb543dfd' args={} name='get_schema' None
None None id='adk-03f904c5-cb65-47f5-9cda-a132eb543dfd' name='get_schema' response={'result': [{'label': 'Person', 'attributes': {'summary': 'STRING', 'id': 'STRING unique indexed', 'name': 'STRING indexed'}, 'relationships': {'HAS_PARENT': 'Person', 'HAS_CHILD': 'Person'}}, {'label': 'Organization', 'attributes': {'summary': 'STRING', 'isDissolved': 'BOOLEAN', 'id': 'STRING unique indexed', 'diffbotId': 'STRING', 'nbrEmployees': 'INTEGER', 'name': 'STRING indexed', 'isPublic': 'BOOLEAN', 'motto': 'STRING', 'revenue': 'FLOAT'}, 'relationships': {'HAS_COMPETITOR': 'Organizatio

'There are 8,064 people in the database.'

In [48]:
await run_prompt('What are the main competitors of YouTube?')

Okay, I can look that up for you. First, I need to find YouTube in the database and then check for its competitors using the `HAS_COMPETITOR` relationship.

 None None
None id='adk-33b02783-a9b5-4378-b4a6-2989b0700052' args={'params': {'name': 'YouTube'}, 'query': 'MATCH (o:Organization {name: $name})-[:HAS_COMPETITOR]-(c:Organization) RETURN c.name AS competitorName'} name='execute_read_query' None
None None id='adk-33b02783-a9b5-4378-b4a6-2989b0700052' name='execute_read_query' response={'result': [{'competitorName': 'Mixer'}, {'competitorName': 'BYTEDANCE'}, {'competitorName': 'BuzzFeed'}, {'competitorName': 'TikTok'}, {'competitorName': 'TikTok'}, {'competitorName': 'OpenAI'}, {'competitorName': 'Dailymotion'}, {'competitorName': 'Twitter'}, {'competitorName': 'Fox Broadcasting'}, {'competitorName': 'Violin Systems'}, {'competitorName': 'Oxygen'}]}
Okay, I found the main competitors for YouTube listed in the database. They are:

*   Mixer
*   BYTEDANCE
*   BuzzFeed
*   TikTok
*   O

'Okay, I found the main competitors for YouTube listed in the database. They are:\n\n*   Mixer\n*   BYTEDANCE\n*   BuzzFeed\n*   TikTok\n*   OpenAI\n*   Dailymotion\n*   Twitter\n*   Fox Broadcasting\n*   Violin Systems\n*   Oxygen'

In [38]:
await run_prompt('What are 5 companies mentioned in articles from January 2023 with good sentiment and who are the people working there?')

Okay, I can search for that. I will look for articles published between January 1st, 2023, and January 31st, 2023, with a positive sentiment (let's define "good" as sentiment greater than 0.5). Then, I'll identify up to 5 distinct companies mentioned in those articles and find the names of people associated with them (CEOs and Board Members based on the schema).

 None None
None id='adk-cf6a89ef-5936-4ace-929c-c2dd8dd89aed' args={'query': '\n    MATCH (a:Article)-[:MENTIONS]-(o:Organization)\n    WHERE a.date >= date($startDate) AND a.date < date($endDate) AND a.sentiment > $minSentiment\n    WITH DISTINCT o\n    LIMIT 5\n    OPTIONAL MATCH (o)-[:HAS_CEO]-(ceo:Person)\n    OPTIONAL MATCH (o)-[:HAS_BOARD_MEMBER]-(bm:Person)\n    WITH o, collect(DISTINCT ceo.name) + collect(DISTINCT bm.name) AS peopleNames \n    WITH o.name AS companyName, [name IN peopleNames WHERE name IS NOT NULL] AS associatedPeople\n    RETURN companyName, associatedPeople\n    ', 'params': {'endDate': '2023-02-01',

ERROR:agent_neo4j_cypher:Database error executing query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid use of DISTINCT with function 'कलेक्ट' (line 9, column 13 (offset: 379))
"    WITH o, कलेक्ट(DISTINCT ceo) + कलेक्ट(DISTINCT bm) AS people"
             ^}

    MATCH (a:Article)-[:MENTIONS]-(o:Organization)
    WHERE a.date >= datetime($startDate) AND a.date < datetime($endDate) AND a.sentiment > $lowerSentiment
    WITH DISTINCT o
    LIMIT 5
    OPTIONAL MATCH (o)-[:HAS_CEO]-(ceo:Person)
    OPTIONAL MATCH (o)-[:HAS_BOARD_MEMBER]-(bm:Person)
    // Collect all people nodes first, then extract names, handling nulls
    WITH o, कलेक्ट(DISTINCT ceo) + कलेक्ट(DISTINCT bm) AS people
    WITH o.name AS companyName, [p IN people WHERE p IS NOT NULL | p.name] AS associatedPeople
    RETURN companyName, associatedPeople
    


Okay, I'll try adjusting the query. Perhaps the sentiment threshold was a bit too high, or the date comparison needs adjustment. I'll lower the sentiment threshold slightly and ensure the date/time comparison is correct.

 None None
None id='adk-d912a8d5-0d21-4fb2-ab84-3cb23eb4ba6e' args={'params': {'startDate': '2023-01-01T00:00:00', 'lowerSentiment': 0.3, 'endDate': '2023-02-01T00:00:00'}, 'query': '\n    MATCH (a:Article)-[:MENTIONS]-(o:Organization)\n    WHERE a.date >= datetime($startDate) AND a.date < datetime($endDate) AND a.sentiment > $lowerSentiment\n    WITH DISTINCT o\n    LIMIT 5\n    OPTIONAL MATCH (o)-[:HAS_CEO]-(ceo:Person)\n    OPTIONAL MATCH (o)-[:HAS_BOARD_MEMBER]-(bm:Person)\n    // Collect all people nodes first, then extract names, handling nulls\n    WITH o, कलेक्ट(DISTINCT ceo) + कलेक्ट(DISTINCT bm) AS people\n    WITH o.name AS companyName, [p IN people WHERE p IS NOT NULL | p.name] AS associatedPeople\n    RETURN companyName, associatedPeople\n    '} name='exe

"Okay, here are 5 companies mentioned in articles with positive sentiment (greater than 0.3) between January 1st and January 31st, 2023, along with people associated with them (CEOs or Board Members found in the database):\n\n1.  **BackBox:**\n    *   Andrew Kahl\n    *   Chris Pacitti\n2.  **Pocket App:**\n    *   Paul Swaddle\n3.  **Mozilla:**\n    *   (No associated people found in the database)\n4.  **Wipro:**\n    *   (No associated people found in the database)\n\nIt seems Wipro appeared twice in the results, but it's the same company. For Mozilla and Wipro, the database did not have information linking specific CEOs or Board Members to them based on the relationships I checked (`HAS_CEO`, `HAS_BOARD_MEMBER`)."

In [49]:
await run_prompt('Who has invested in BYTEDANCE and where else have they invested?')

Okay, I can help you with that request. I'll first find the investors for BYTEDANCE and then look for other companies those same investors have put money into.

Let's find the investors for BYTEDANCE first.

 None None
None id='adk-ba9ce6de-2afd-4956-9347-6d549d655cf1' args={'params': {'name': 'BYTEDANCE'}, 'query': 'MATCH (o:Organization {name: $name})-[:HAS_INVESTOR]-(investor:Person) RETURN investor.name AS investorName, investor.id AS investorId'} name='execute_read_query' None
None None id='adk-ba9ce6de-2afd-4956-9347-6d549d655cf1' name='execute_read_query' response={'result': [{'investorName': 'Rong Yue', 'investorId': 'EHwJNEbHuN9S0OtngP6cFKQ'}, {'investorName': 'Wendi Murdoch', 'investorId': 'En8xLxQ3oNnWcR6ZwsWol3w'}]}
Okay, I found two investors for BYTEDANCE: Rong Yue and Wendi Murdoch.

Now, let's see where else these individuals have invested.

 None None
None id='adk-15b1e3d1-0af0-4f88-8c62-de879a43f444' args={'query': 'MATCH (investor:Person)-[:HAS_INVESTOR]-(otherCompan

'Okay, I found the investors for BYTEDANCE and their other investments:\n\n*   **Rong Yue** invested in BYTEDANCE and also in **Inspur**.\n*   **Wendi Murdoch** invested in BYTEDANCE. According to the database, there are no other investments listed for Wendi Murdoch.'

In [50]:
runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)

ListSessionsResponse(sessions=[Session(id='3375d095-3f4d-4725-97c6-9f7318c6574a', app_name='Neo4j Investment Researcher', user_id='Michael Hunger', state={}, events=[], last_update_time=1744237962.688923)])

In [51]:
for session in runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID).sessions:
  print(session.model_dump())

{'id': '3375d095-3f4d-4725-97c6-9f7318c6574a', 'app_name': 'Neo4j Investment Researcher', 'user_id': 'Michael Hunger', 'state': {}, 'events': [], 'last_update_time': 1744237962.688923}


In [52]:
result = await run_prompt("Summarize the results of the previous research questions")
result

Okay, here's a summary of our findings so far:

1.  We first determined that there are **8,064 people** represented in the database.
2.  Next, we looked up the competitors for **YouTube** and found several, including Mixer, BYTEDANCE, BuzzFeed, TikTok, OpenAI, Dailymotion, Twitter, Fox Broadcasting, Violin Systems, and Oxygen.
3.  Finally, we identified the investors in **BYTEDANCE** as **Rong Yue** and **Wendi Murdoch**. We also found that Rong Yue has invested in **Inspur**, while no other investments were listed for Wendi Murdoch in the database besides BYTEDANCE. None None


"Okay, here's a summary of our findings so far:\n\n1.  We first determined that there are **8,064 people** represented in the database.\n2.  Next, we looked up the competitors for **YouTube** and found several, including Mixer, BYTEDANCE, BuzzFeed, TikTok, OpenAI, Dailymotion, Twitter, Fox Broadcasting, Violin Systems, and Oxygen.\n3.  Finally, we identified the investors in **BYTEDANCE** as **Rong Yue** and **Wendi Murdoch**. We also found that Rong Yue has invested in **Inspur**, while no other investments were listed for Wendi Murdoch in the database besides BYTEDANCE."

In [53]:
from IPython.display import Markdown, display


display(Markdown(result))

Okay, here's a summary of our findings so far:

1.  We first determined that there are **8,064 people** represented in the database.
2.  Next, we looked up the competitors for **YouTube** and found several, including Mixer, BYTEDANCE, BuzzFeed, TikTok, OpenAI, Dailymotion, Twitter, Fox Broadcasting, Violin Systems, and Oxygen.
3.  Finally, we identified the investors in **BYTEDANCE** as **Rong Yue** and **Wendi Murdoch**. We also found that Rong Yue has invested in **Inspur**, while no other investments were listed for Wendi Murdoch in the database besides BYTEDANCE.

In [54]:
for session in runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID).sessions:
  print(f"Deleting session {session}")
  runner.session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=session.id)

Deleting session id='3375d095-3f4d-4725-97c6-9f7318c6574a' app_name='Neo4j Investment Researcher' user_id='Michael Hunger' state={} events=[] last_update_time=1744237973.959884
